In [ ]:
import twint
import nest_asyncio
nest_asyncio.apply()

In [ ]:
usuario =['railanderReis','RapMais']
data_init = '2010-01-01'

In [ ]:
for i in usuario:    
    c = twint.Config()
    c.Username = i
    c.Since = data_init
    c.Store_csv = True
    c.Output = (i+".csv")
    twint.run.Search(c)

In [1]:
import tweepy
import re
import pandas as pd
import numpy as np
from datetime import datetime
import json
import glob2
import pymongo

In [2]:
consumer_key = 'yhJM02AXVkeJKbZ2UY9MZx2Pk'
consumer_secret = 'bLNfYOuu0HJsca70Em3TQxoOAdudra0jsZI8kf1Rb7L74AxAjZ'
access_token = '164492542-o5pBhnoeyeNr4lsW1soVsGAWA6h8qH7k9E2j4I9t'
access_token_secret = 'S5nXv7sJZaC348zfHEzTcNK4Q1BAOUsOi6KLBH6YlTM0m'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [4]:
usuario =['railanderReis','RapMais']


In [ ]:
for i in usuario:
    try:     
        tweets = tweepy.Cursor(api.user_timeline,id=i).items(30)
        
        tweets_list = [[tweet.user.screen_name, tweet.text,tweet.created_at] for tweet in tweets]
                
        tweets_df = pd.DataFrame(tweets_list)
        tweets_df = tweets_df.rename(columns={0: 'User', 1: 'Tweets', 2: 'Post_date'})

        tweets_df['Tweets'] = tweets_df['Tweets'].replace(to_replace=r'#\S+', value='', regex=True)
        tweets_df['Tweets'] = tweets_df['Tweets'].replace(to_replace=r'@\S+', value='', regex=True)
        
        tweets_df['Tweets'] = tweets_df['Tweets'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

        tweets_df['Tweets'] = tweets_df['Tweets'].replace(to_replace=r'\$\w*', value='', regex=True)
        tweets_df['Tweets'] = tweets_df['Tweets'].replace(to_replace=r'https?:\/\/.*[\r\n]*', value='', regex=True)
        tweets_df['Tweets'] = tweets_df['Tweets'].replace(to_replace=r'\n', value='', regex=True)
        tweets_df['Tweets'] = tweets_df['Tweets'].replace(to_replace=r'RT', value='', regex=True)
        tweets_df['Tweets'] = tweets_df['Tweets'].replace(to_replace=r'\""', value='', regex=True)
        tweets_df['Tweets'] = tweets_df['Tweets'].str.upper().str.lstrip()
                

        tweets_df['Post_date'] = pd.to_datetime(tweets_df['Post_date'], format='%d-%m-Y',infer_datetime_format=True)    

        tweets_df.to_csv(i+".csv", index = False, header=True)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)

In [5]:
for i in usuario:
    try:     
        tweets = tweepy.Cursor(api.user_timeline,id=i).items(30)
        
        tweets_list = [[tweet.user.screen_name, tweet.text,tweet.created_at] for tweet in tweets]
                
        tweets_df = pd.DataFrame(tweets_list)
        tweets_df = tweets_df.rename(columns={0: 'User', 1: 'Tweets', 2: 'Post_date'})

        tweets_df.to_csv(i+".csv", index = False, header=True)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)

In [6]:

path =r'F:/cods-dockers/astro/dags'
allFiles = glob2.glob(path + "/*.csv")



In [14]:

#stockstats_data = pd.DataFrame()
list_ = []

for file_ in allFiles:
    dfs = pd.read_csv(file_,index_col=None,header=0)
    list_.append(dfs)
    transform_data = pd.concat(list_,axis=0, ignore_index=True)
    

In [8]:
transform_data.head(2)

,User,Tweets,Post_date
0,railanderReis,@Reisrz Ou nhm convite pra rock...😂😂😂,2021-04-17 20:38:57
1,railanderReis,"@Reisrz Conheco, mas tenho que confirmar se el...",2021-03-04 21:59:27


In [9]:
transform_data['Tweets'] = transform_data['Tweets'].replace(to_replace=r'https?:\/\/.*[\r\n]*', value='', regex=True)
transform_data['Tweets'] = transform_data['Tweets'].replace(to_replace=r'#\S+', value='', regex=True)
transform_data['Tweets'] = transform_data['Tweets'].replace(to_replace=r'@\S+', value='', regex=True)
transform_data['Tweets'] = transform_data['Tweets'].replace(to_replace=r'\$\w*', value='', regex=True)
transform_data['Tweets'] = transform_data['Tweets'].replace(to_replace=r'\n', value='', regex=True)
transform_data['Tweets'] = transform_data['Tweets'].replace(to_replace=r'\""', value='', regex=True)
transform_data['Tweets'] = transform_data['Tweets'].replace(to_replace=r'  ', value=' ', regex=True)
transform_data['Tweets'] = transform_data['Tweets'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
transform_data['Tweets'] = transform_data['Tweets'].str.upper().str.lstrip().str.strip()

                
transform_data['Post_date'] = pd.to_datetime(transform_data['Post_date'], format='%d-%m-Y',infer_datetime_format=True) 
transform_data['Post_date'] = transform_data['Post_date'].dt.strftime('%Y-%m-%d')

transform_data.insert(3, "Upload_date",datetime.today().strftime('%Y-%m-%d'), allow_duplicates=False)
       

In [10]:
transform_data.head(10)

,User,Tweets,Post_date,Upload_date
0,railanderReis,OU NHM CONVITE PRA ROCK...,2021-04-17,2021-04-25
1,railanderReis,"CONHECO, MAS TENHO QUE CONFIRMAR SE ELA ESTA P...",2021-03-04,2021-04-25
2,RapMais,RT MIRANDO UM FEAT FUTURO COM TRAVIS SCOTT EL...,2021-04-25,2021-04-25
3,RapMais,RT HOJE ESTAREI PARTICIPANDO DO PROGRAMA PARA ...,2021-04-25,2021-04-25
4,RapMais,POLICIAL QUE MATOU GEORGE FLOYD ESTÁ SENDO INV...,2021-04-24,2021-04-25
5,RapMais,O RAPPER KANYE WEST ESTÁ NESTE MOMENTO COM O C...,2021-04-24,2021-04-25
6,RapMais,6IX9INE FAZ PRIMEIRO SHOW DESDE QUE SAIU DA PR...,2021-04-24,2021-04-25
7,RapMais,PK DELAS DEBOCHA APÓS SER DETIDO POR FAZER SEX...,2021-04-24,2021-04-25
8,RapMais,RAPPER DMX GRAVOU SEU ÚLLTIMO ÁLBUM NO ESTÚDIO...,2021-04-24,2021-04-25
9,RapMais,TA.INC LIBERA VIDEOCLIPE DO SINGLE GLOCK NA CI...,2021-04-24,2021-04-25


In [11]:
url = "mongodb://127.0.0.1:27017/"
con = pymongo.MongoClient(url)

db = con.twitterdb

collection = db.tweets_famosos



In [12]:
transform_data.reset_index(inplace=True) # Reset Index
data_dict = transform_data.to_dict("records") # Convert to dictionary
db.tweets_famosos.insert_many(data_dict) # inesrt into DB

In [13]:
db.tweets_famosos.count_documents({})

62

In [ ]:
data_atual = datetime.today().strftime('%Y-%m-%d')
print(data_atual)